# Tasks

_Feature eng_
- user, prod, user-prod
- category (aisle)
- timeslot-prod

_Training with xgboost_
- search hyper-params
  - **approx splitting, loss guided growing**
  - lr schedule
- threshold setting
- test importance of random feature

_Advanced feature eng_
- customer segmentation - 根据user-aisle购买比例作为每个user的feature来做clustering，形成segment
  - s feature 把用户分类成segment
  - sp feature
- **data augmentation**
  - create "augment/fake users" which have a copied subset version of orders from a real user

_Discussion_
- **None handling**
    - https://www.kaggle.com/c/instacart-market-basket-analysis/discussion/35716
    - https://www.kaggle.com/c/instacart-market-basket-analysis/discussion/37016
    - https://www.kaggle.com/c/instacart-market-basket-analysis/discussion/36848
- **optimize F1 score**
    - https://www.kaggle.com/c/instacart-market-basket-analysis/discussion/37221
    - https://arxiv.org/pdf/1206.4625.pdf
    - https://www.kaggle.com/mmueller/f1-score-expectation-maximization-in-o-n/code
- collaborative filtering

# Feature engineering

```
priors.dtype={
                'order_id': np.int32,
                'product_id': np.uint16,
                'add_to_cart_order': np.int16,
                'reordered': np.int8})
train.dtype={
                'order_id': np.int32,
                'product_id': np.uint16,
                'add_to_cart_order': np.int16,
                'reordered': np.int8})
orders.dtype={
                'order_id': np.int32,
                'user_id': np.int64,
                'eval_set': 'category',
                'order_number': np.int16,
                'order_dow': np.int8,
                'order_hour_of_day': np.int8,
                'days_since_prior_order': np.float32})
products.dtype={
                'product_id'
                'product_name'
                'aisle_id'
                'department_id'}
```

```
priors_orders_detail = orders.merge(priors, how='inner', on='order_id')
_user_buy_product_times: 用户在此单（包括）之前一共买过几次该商品
```

## Product feature

- `_prod_total_cnt`: 商品被买的总次数

reorder stats
- `_prod_reorder_total_cnt`: 商品被reorder的总次数
- `_prod_user_cnt`: 有多少人买过这个商品
- `_prod_return_user_cnt`: 有多少人第二次买这个商品
- `_prod_user_reorder_ratio`: 有百分之多少的人第一次买了还买了第二次
- `_prod_product_reorder_ratio`: 商品被reorder的总次数/商品被买的总次数

~~`_prod_reorder_times`~~


## User feature

counts
- `_user_total_orders`: order数，用户下的prior总单数
- `_user_total_products`: 商品数，用户从第一单到最后一单prior的总商品数
- `_user_distinct_products`: unique商品数，用户从第一单到最后一单prior的unique商品数

rates
- `_user_mean_days_since_prior_order`: 下单频率，用户从第一单到最后一单prior的平均间隔天数
- `_user_reorder_ratio`: reorder比例，从第二单到最后一单prior，用户reorder的商品比例
- `_user_average_basket`: 平均cart大小，总商品数/总单数

timeline
- `_user_sum_days_since_prior_order`: cart龄，用户从第一单到最后一单prior的总天数间隔
- `time_since_last_order`: user的最后一单(train/test)距离最后一单prior的天数


## User-Product feature


timeline
- `_up_first_order_number`: 该用户买该商品的第一单
- `_up_last_order_number`: 该用户买该商品的最后一单
- `_up_order_since_last_order`: 用户下的prior总单数-该用户买该商品的最后一单
- [] days since last order

order rate
- `_up_order_rate`: 该用户买该商品次数/用户下的prior总单数
- `_up_order_rate_since_first_order`: 该用户买该商品次数/(用户下的prior总单数 - 该用户买该商品的第一单 + 1)

others
- `_up_order_count`: 该用户买该商品次数
- `_up_average_cart_position`: 该用户买该商品的平均加入cart位置

- ~~[] recent orders最近用户是不是买过该产品（最近五单/最近30天）~~


## Category feature

- `cat_total_bought_cnts`
- `cat_total_reorder_cnts`
- `cat_user_cnt`
- `cat_return_user_cnt`
- `cat_user_reorder_ratio`
- `cat_reorder_ratio`
- `cat_num_of_prods_a_user_buys_in_this_cat_(mean/std/max/median)`

## Timeslot-Product feature

- for a given product, what is the "market share" of each timeslot as compared with other timeslots